链家网数据爬取

参考博客：https://blog.csdn.net/qq_46256922/article/details/119087591

In [1]:
import pandas as pd # 数据存储
import requests # 网页内容获取
import re # 解析数据
from lxml import etree # 解析数据
import random 
import time # 反反爬
from fastprogress import master_bar,progress_bar # 进度条显示

In [2]:
def ua():
    """随机获取一个浏览器用户信息"""
    
    # https://blog.csdn.net/BobYuan888/article/details/88950275
    user_agents = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
    'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',
    ]

    agent = random.choice(user_agents)
    
    return {
        'User-Agent': agent
    }


def get(url):
    """
    获取网页源码
    url: 目标网页的地址
    return:网页源码
    """
    res = requests.get(url=url, headers = ua())
    return res.text

def get_url(res_text):
    """
    获取源码中每个二手房详情页的url
    res_text:网页源码
    return:列表形式的30个二手房详情页的url
    """
    re_f = '<a class="" href="(.*?)" target="_blank"'
    url_list = re.findall(re_f, res_text)
    return url_list


In [3]:
def get_data(res_text):
    """
    获取房屋的详细数据
    etree.HTML()
    调用HTML类对HTML文本进行初始化，成功构造XPath解析对象，同时可以自动修正HMTL文本（标签缺少闭合自动添加上）
    etree.tostring()
    tostring()方法可以输出修正之后的HTML代码，也可以直接读取文本进行解析，但是结果为bytes类型，因此需要利用decode()方法将其转成str类型
    """
    res_text = etree.HTML(res_text)
    # https://blog.csdn.net/Beyond_F4/article/details/89311276
#     print(etree.tostring(res_text).decode('utf-8'))

    '''
    xml: https://www.w3school.com.cn/xml/xml_tree.asp
    xpath: https://www.w3school.com.cn/xpath/xpath_syntax.asp
    text(): https://jingyan.baidu.com/article/b87fe19eac63cd1318356886.html
    /与//的区别：https://www.cnblogs.com/myboat/p/13953114.html
    '''
    # 获取房屋的标题
    title = res_text.xpath("//div[@class='sellDetailHeader']//h1/@title")
#     ff = title[0].split(" ")
#     print(ff)
#     print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # 获取房屋的总价
    total_price = res_text.xpath("//div[@class='overview']//div/span/text()")[2]
#     print(total_price)
    # 获取房屋的单价
    price = res_text.xpath("//div[@class='overview']//div/span/text()")[3]
    # 获取房屋的地段
    communityName = res_text.xpath("//div[@class='aroundInfo']//div/a/text()")[0]
#     print(communityName)
    
    regionName = res_text.xpath("//div[@class='areaName']//span/a/text()")[0]
#     print(regionName)
    # 获取区域
    areaName = res_text.xpath("//div[@class='areaName']//span/a/text()")[1]
#     print(areaName)
    
    # 简精装
    
    ## 房屋基本信息获取
    # 获取房屋基本信息的标题
    lab = res_text.xpath("//div[@class='base']//span/text()")
#     print(lab)
    # 获取房屋基本信息的内容
    val = res_text.xpath("//div[@class='base']//li/text()")
#     print(val)
    
    ## 获取房源交易信息
    # 获取房源交易标题
    key1 = res_text.xpath("//div[@class='transaction']//span[1]//text()")
    # 获取房源交易信息内容
    trans = res_text.xpath("//div[@class='transaction']//span[2]//text()")
    
    # 返回包含上述信息的字典
    return dict(zip(['标题', '总价格', '单价', '小区','房屋所属市辖区','房屋地址（街道）'] + lab + key1 ,
                    [title, total_price, price, communityName,regionName,areaName] + val + trans))

In [4]:
def main(qu,start_pg=1, end_pg=100, download_times=1):
    """爬虫程序
    qu: 传入要爬取的区的拼音的列表
    start_pg:开始的页码
    end_pg:结束的页码
    download_times:第几次下载
    """
    for q in qu:
        # 获取链家网（绵阳）的首页url,同学们可以换成自己想要的城市url
        url = 'https://cd.lianjia.com/ershoufang/' + q + '/pg'
        # 数据储存的列表
        data = []
        # 文件保存路径
        filename = './data/二手房-' + q +'第' + str(download_times) + '次下载.csv'
#         print(filename)
        
        print('二手房-' + q +'第' + str(download_times) + '次下载')
        mb = master_bar(range(start_pg, end_pg+1))
    
        for i in mb:
            
            # 获取每页的url
            new_url = url + str(i) + '/'
#             print(new_url)
            
            # 获取当前页面包含的30个房屋详情页的url
            url_list = get_url(get(new_url))

            for l in progress_bar(range(len(url_list)),parent=mb):
                
                # 反爬随机停止一段时间
                a = random.randint(2, 5)
                if l % a == 0:
                    time.sleep(2 * random.random())
                try:
                    # 获取当前页面的源码
                    text = get(url_list[l])
                    # 获取当前页面的房屋信息
                    data.append(get_data(text))
                except Exception as e:
                    time.sleep(100 * random.random())
                    # 获取当前页面的源码
                    text = get(url_list[l])
                    # 获取当前页面的房屋信息
                    data.append(get_data(text))
                    
                # 反爬随机停止一段时间
                time.sleep(5 * random.random())
                mb.child.comment = '正在爬取第'+str(l + 1) +'条数据!!'
            mb.main_bar.comment = '正在爬取第'+str(i+1) +'页数据!!'
            
            # 反爬随机停止一段时间
            time.sleep(5 * random.random())
            
            if i % 5 == 0:
                # 每5页保存一次数据
                pd.DataFrame(data).to_csv(filename,encoding="utf_8_sig")
                mb.write('前'+str(i)+'页数据已保存')
    pd.DataFrame(data).to_csv(filename,encoding="utf_8_sig")

In [5]:
# 例如
main(['tianfu'],1,100,1) # 区，开始页，结束页，第几次下载（以便保存文件）

二手房-doujiangyan第1次下载


IndexError: list index out of range